In [1]:
import torch
import torch.nn.functional as F
from torch_geometric.data import DataLoader
from torch_geometric.nn import GraphSAGE
import torch_geometric.transforms as T
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
import numpy as np
from tqdm import tqdm
from torch_geometric.data import Data
from torch_geometric.nn import SAGEConv
from sklearn.utils import shuffle

from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

import ipysigma

/Users/jskaf/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')

In [3]:
G = nx.Graph()
G_plot = nx.Graph()

In [4]:
iris = load_iris()
data = np.hstack((iris['data'],[[f'Gerard {i + 1}'] for i in range(len(iris['data']))]))
y_true = iris['target']
y_true_tensor = torch.tensor(y_true, dtype=torch.long, requires_grad=False)
columns_mins = torch.tensor(np.min(iris['data'], axis=0), requires_grad=False, dtype=torch.float32)
columns_maxs = torch.tensor(np.max(iris['data'], axis=0), requires_grad=False, dtype=torch.float32)
columns_mins

tensor([4.3000, 2.0000, 1.0000, 0.1000])

In [5]:
similarity_matrix = cosine_similarity(iris['data'],iris['data'])
minimum_similarity = 0.9

In [6]:
for i in range(len(similarity_matrix)):
    if "Gerard " + str(i + 1) not in G_plot.nodes:
        G_plot.add_node("Gerard " + str(i + 1), sepal_length=iris['data'][i][0], sepal_width=iris['data'][i][1],
                   petal_length=iris['data'][i][2], petal_width=iris['data'][i][3])
        G.add_node(i, sepal_length=iris['data'][i][0], sepal_width=iris['data'][i][1],
                   petal_length=iris['data'][i][2], petal_width=iris['data'][i][3])
    for j in range(i):
        if "Gerard " + str(j + 1) not in G.nodes:
            G_plot.add_node("Gerard " + str(j + 1), sepal_length=iris['data'][j][0], sepal_width=iris['data'][j][1],
                       petal_length=iris['data'][j][2], petal_width=iris['data'][j][3])
            G.add_node(j, sepal_length=iris['data'][j][0], sepal_width=iris['data'][j][1],
                       petal_length=iris['data'][j][2], petal_width=iris['data'][j][3])
        if similarity_matrix[i][j] >= minimum_similarity:
            G_plot.add_edge("Gerard " + str(i + 1), "Gerard " + str(j + 1), weight=similarity_matrix[i][j])
            G.add_edge(i, j, weight=similarity_matrix[i][j])

print(nx.density(G))
print(nx.density(G_plot))

0.8083221476510067
0.8083221476510067


In [7]:
x = torch.tensor(iris['data'], dtype=torch.float)
edges = list(G.edges)

edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
data_target_model = Data(x=x, edge_index=edge_index)
data_target_model.y = y_true_tensor
data_target_model.validate(raise_on_error=True)

True

In [8]:
# Manually split nodes into train, validation, and test sets using PyTorch
num_nodes = len(data_target_model.x)
train_ratio, test_ratio = 0.8, 0.2

num_train = int(train_ratio * num_nodes)
num_test = num_nodes - num_train

# Create masks for train, validation, and test nodes
perm = torch.randperm(num_nodes)
train_mask = perm[:num_train]
test_mask = perm[num_train:]

# Apply masks to the data
data_target_model.train_mask = torch.zeros(num_nodes, dtype=torch.bool)
data_target_model.test_mask = torch.zeros(num_nodes, dtype=torch.bool)

data_target_model.train_mask[train_mask] = 1
data_target_model.test_mask[test_mask] = 1

In [9]:
class GraphSAGE(torch.nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim, dropout=0.2):
        super().__init__()
        self.conv1 = SAGEConv(in_dim, hidden_dim)
        self.conv3 = SAGEConv(hidden_dim, out_dim)
    
    def forward(self, data):
        x = self.conv1(data.x, data.edge_index)
        x = F.elu(x)
        
        x = self.conv3(x, data.edge_index)
        x = F.elu(x)
        return x

In [10]:
target_model = GraphSAGE(data_target_model.num_features, hidden_dim=16, out_dim=3)
optimizer = torch.optim.Adam(target_model.parameters(), lr=0.01)

In [11]:
# Training loop
target_model.train()
for epoch in range(201):
    optimizer.zero_grad()
    out = target_model(data_target_model)
    loss = F.cross_entropy(out[data_target_model.train_mask], y_true_tensor[data_target_model.train_mask])
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print(f'Epoch: {epoch:03d}, Train Loss: {loss:.3f}')

# Evaluation
target_model.eval()
with torch.no_grad():
    logits = target_model(data_target_model)
    pred = logits.argmax(dim=1)

# Evaluate accuracy
accuracy = (pred[data_target_model.test_mask] == y_true_tensor[data_target_model.test_mask]).sum().item() / data_target_model.test_mask.sum().item()
print(f"Accuracy: {accuracy:.4f}")

Epoch: 000, Train Loss: 1.335
Epoch: 010, Train Loss: 0.765
Epoch: 020, Train Loss: 0.312
Epoch: 030, Train Loss: 0.126
Epoch: 040, Train Loss: 0.056
Epoch: 050, Train Loss: 0.030
Epoch: 060, Train Loss: 0.020
Epoch: 070, Train Loss: 0.016
Epoch: 080, Train Loss: 0.013
Epoch: 090, Train Loss: 0.011
Epoch: 100, Train Loss: 0.010
Epoch: 110, Train Loss: 0.009
Epoch: 120, Train Loss: 0.008
Epoch: 130, Train Loss: 0.007
Epoch: 140, Train Loss: 0.006
Epoch: 150, Train Loss: 0.006
Epoch: 160, Train Loss: 0.005
Epoch: 170, Train Loss: 0.005
Epoch: 180, Train Loss: 0.004
Epoch: 190, Train Loss: 0.004
Epoch: 200, Train Loss: 0.004
Accuracy: 0.9667


y_pred = torch.argmax(torch.nn.functional.softmax(model.forward(data)), axis=1).detach().numpy()
ipysigma.Sigma(G, node_color=y_pred)

In [17]:
def generate_synthetic_data(k_max, c, target_model,columns_mins,columns_maxs, iterations=1000, conf_min=0.99, rej_max=1000, k_min = 1, old_x_k=None):
    j = 0
    if old_x_k is not None:
        x = old_x_k + torch.rand(1)
    else:
        x = torch.tensor(torch.rand(1, len(columns_mins)) * (columns_maxs.detach() - columns_mins.detach()) + columns_mins.detach(), requires_grad=False, dtype=torch.float)
    edges = [(0,0)]
    edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
    data = Data(x=x, edge_index=edge_index)
    y_c = 0
    k = k_max
    for i in range(iterations):
        y_class = torch.nn.functional.softmax(target_model.forward(data)).detach()
        if y_class[0][c].item() > y_c:
            if y_class[0][c].item() > conf_min and c == torch.argmax(y_class[0]).item():
                if np.random.uniform(0, 1) <= y_class[0][c].item():
                   return x
            x_temp = x
            y_c = y_class[0][c].item()
            j = 0
        else:
            j += 1
            if j>=rej_max:
                k = max(k_min, k//2)
                j = 0
        indice = np.random.randint(0, 4, size=(1, k))
        for i in indice:
            x[0, i] = torch.rand(1) * (columns_maxs[i] - columns_mins[i]) + columns_mins[i]
    return None

In [18]:
def generate_shadows_datasets(k_max, c, target_model, columns_mins, columns_maxs, mutliple_of_cs, iterations=100, conf_min=0.99, rej_max=100000, k_min = 1, n=10000, apriori = False):
    shadow_dataset = []
    old_x_k = None
    nb_used_a_priori = 0
    print(f'Generating {c} label shadow...')
    for j in range(mutliple_of_cs):
        for i in tqdm(range(n)):
            x_k = generate_synthetic_data(k_max, c, target_model, columns_mins, columns_maxs, iterations, conf_min, rej_max, k_min, old_x_k)
            while x_k is None:
                x_k = generate_synthetic_data(k_max, c, target_model, columns_mins, columns_maxs, iterations, conf_min, rej_max, k_min, old_x_k)
            shadow_dataset.append(x_k)
            if apriori and nb_used_a_priori < 50:
                old_x_k = x_k
                nb_used_a_priori += 1
            if nb_used_a_priori >= 50:
                for i in range(len(old_x_k[0])):
                    if x[0][i] > 0.80 * columns_maxs[i]:
                        x[0][i] = torch.rand(1) * (columns_maxs[i] - columns_mins[i]) + columns_mins[i]
                    if x[0][i] < 1.20 * columns_mins[i]:
                        x[0][i] = torch.rand(1) * (columns_maxs[i] - columns_mins[i]) + columns_mins[i]
                        nb_used_a_priori = 0
    print(f'{c} label shadow dataset generated\n')
    return shadow_dataset

In [12]:
c_s = [0, 1, 2]
mutliple_of_cs = 1
n = 10000

In [213]:
shadow_dataset_0 = generate_shadows_datasets(4, 0, target_model, columns_mins, columns_maxs, mutliple_of_cs=mutliple_of_cs, n=n)

Generating 0 label shadow...


  0%|          | 0/10000 [00:00<?, ?it/s]/var/folders/62/6pb5kjf139gccp8_mvcxrc7w0000gn/T/ipykernel_88484/2410329687.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(torch.rand(1, len(columns_mins)) * (columns_maxs.detach() - columns_mins.detach()) + columns_mins.detach(), requires_grad=False, dtype=torch.float)
/var/folders/62/6pb5kjf139gccp8_mvcxrc7w0000gn/T/ipykernel_88484/2410329687.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  y_class = torch.nn.functional.softmax(target_model.forward(data)).detach()
100%|██████████| 10000/10000 [00:58<00:00, 170.43it/s]

0 label shadow dataset generated



In [211]:
shadow_dataset_1 = generate_shadows_datasets(4, 1, target_model, columns_mins, columns_maxs, mutliple_of_cs=mutliple_of_cs, n=n, apriori=True)

Generating 1 label shadow...


  0%|          | 0/10000 [00:00<?, ?it/s]/var/folders/62/6pb5kjf139gccp8_mvcxrc7w0000gn/T/ipykernel_88484/2410329687.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(torch.rand(1, len(columns_mins)) * (columns_maxs.detach() - columns_mins.detach()) + columns_mins.detach(), requires_grad=False, dtype=torch.float)
/var/folders/62/6pb5kjf139gccp8_mvcxrc7w0000gn/T/ipykernel_88484/2410329687.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  y_class = torch.nn.functional.softmax(target_model.forward(data)).detach()
100%|██████████| 10000/10000 [29:26<00:00,  5.66it/s] 

1 label shadow dataset generated



In [212]:
shadow_dataset_1

[tensor([[7.4407, 4.0938, 4.1817, 0.3805]]),
 tensor([[7.7382, 2.9787, 3.4059, 0.1346]]),
 tensor([[8.0005, 3.2410, 3.6682, 0.3969]]),
 tensor([[8.0035, 3.2440, 3.6712, 0.3999]]),
 tensor([[6.8349, 3.6899, 4.1251, 0.1441]]),
 tensor([[6.8694, 3.7244, 4.1596, 0.1786]]),
 tensor([[7.5295, 2.1907, 2.5414, 0.2907]]),
 tensor([[7.5588, 2.2200, 2.5707, 0.3200]]),
 tensor([[7.5674, 2.2287, 2.5793, 0.3287]]),
 tensor([[7.5702, 2.2314, 2.5821, 0.3314]]),
 tensor([[7.6107, 3.1238, 3.7628, 0.2083]]),
 tensor([[7.6195, 3.1326, 3.7715, 0.2171]]),
 tensor([[7.6400, 3.1531, 3.7920, 0.2376]]),
 tensor([[7.6417, 3.1548, 3.7937, 0.2392]]),
 tensor([[7.8357, 2.8715, 3.1423, 0.4333]]),
 tensor([[7.8061, 4.3374, 4.2801, 0.5236]]),
 tensor([[7.6447, 2.1040, 2.4796, 0.2040]]),
 tensor([[7.6559, 2.1152, 2.4908, 0.2152]]),
 tensor([[7.7924, 2.2517, 2.6272, 0.3517]]),
 tensor([[7.8166, 2.2759, 2.6515, 0.3759]]),
 tensor([[6.9838, 3.7892, 3.9241, 0.2448]]),
 tensor([[7.0670, 3.8724, 4.0074, 0.3280]]),
 tensor([[

In [214]:
shadow_dataset_2 = generate_shadows_datasets(4, 2, target_model, columns_mins, columns_maxs, mutliple_of_cs=mutliple_of_cs, n=n)

Generating 2 label shadow...


  0%|          | 0/10000 [00:00<?, ?it/s]/var/folders/62/6pb5kjf139gccp8_mvcxrc7w0000gn/T/ipykernel_88484/2410329687.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(torch.rand(1, len(columns_mins)) * (columns_maxs.detach() - columns_mins.detach()) + columns_mins.detach(), requires_grad=False, dtype=torch.float)
/var/folders/62/6pb5kjf139gccp8_mvcxrc7w0000gn/T/ipykernel_88484/2410329687.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  y_class = torch.nn.functional.softmax(target_model.forward(data)).detach()
100%|██████████| 10000/10000 [00:04<00:00, 2499.86it/s]


2 label shadow dataset generated



In [168]:
shadow_dataset_2

[tensor([[4.7182, 2.6255, 6.2058, 1.2701]]),
 tensor([[6.6992, 2.9685, 5.1157, 1.8997]]),
 tensor([[5.3722, 2.7418, 4.9675, 0.2015]]),
 tensor([[4.5514, 3.6863, 4.7075, 0.4862]]),
 tensor([[7.6488, 3.3724, 2.8565, 2.4919]]),
 tensor([[6.7497, 3.5584, 6.8083, 1.2461]]),
 tensor([[6.4513, 3.8199, 4.5257, 1.5342]]),
 tensor([[7.0234, 3.0461, 5.4633, 1.1461]]),
 tensor([[7.5320, 3.3989, 4.2305, 2.4105]]),
 tensor([[7.3912, 2.0126, 5.4744, 1.8869]]),
 tensor([[4.9614, 4.0071, 6.0164, 0.7743]]),
 tensor([[6.4755, 3.6673, 5.4055, 2.0498]]),
 tensor([[4.8933, 2.0819, 3.3170, 2.0508]]),
 tensor([[5.9814, 3.4564, 6.6045, 1.6933]]),
 tensor([[6.4662, 3.3526, 5.0271, 0.6604]]),
 tensor([[5.8761, 4.3411, 6.2562, 1.7244]]),
 tensor([[7.8528, 2.9303, 6.7533, 1.3216]]),
 tensor([[5.3262, 4.2759, 5.6267, 2.0646]]),
 tensor([[7.0043, 4.0061, 3.9252, 1.5321]]),
 tensor([[5.6339, 2.0574, 5.9909, 2.4181]]),
 tensor([[5.4104, 3.1220, 3.6479, 1.1419]]),
 tensor([[7.3341, 2.2283, 5.1489, 1.0267]]),
 tensor([[

In [143]:
shadow_datasets = [shadow_dataset_0, shadow_dataset_1, shadow_dataset_2]
torch.save(shadow_datasets, 'shadow_datasetsV0.pt')

NameError: name 'shadow_dataset_0' is not defined

In [13]:
shadow_datasets = torch.load('shadow_datasetsV0.pt')

In [14]:
shadow_datasets_train_class_k = []
shadow_datasets_test_class_k = []
shadow_train_labels_class_k = []
shadow_test_labels_class_k = []
for k in c_s:
    temp_train_class_k, temp_test_class_k, temp_train_labels_class_k, temp_test_labels_class_k = train_test_split(shadow_datasets[k], np.full(len(shadow_datasets[k]),c_s[k]), test_size=0.5, shuffle=True)
    shadow_datasets_train_class_k.append(temp_train_class_k)
    shadow_datasets_test_class_k.append(temp_test_class_k)
    shadow_train_labels_class_k.append(temp_train_labels_class_k)
    shadow_test_labels_class_k.append(temp_test_labels_class_k)
shadow_datasets_train_class_k = np.array(shadow_datasets_train_class_k)
shadow_datasets_test_class_k = np.array(shadow_datasets_test_class_k)
shadow_train_labels_class_k = np.array(shadow_train_labels_class_k)
shadow_test_labels_class_k = np.array(shadow_test_labels_class_k)

shadow_test_labels_class_k[2]

array([2, 2, 2, ..., 2, 2, 2])

In [15]:
shadow_dataset_train_k = []
shadow_dataset_test_k = []
shadow_train_labels_k= []
shadow_test_labels_k= []
range_indices_train = len(shadow_datasets_train_class_k[0])//mutliple_of_cs
range_indices_test = len(shadow_datasets_test_class_k[0])//mutliple_of_cs
for k in range(mutliple_of_cs):
    shadow_dataset_train_k.append(np.concatenate([shadow_datasets_train_class_k[i][k*range_indices_train:(k+1)*range_indices_train] for i in range(len(c_s))]))
    shadow_dataset_test_k.append(np.concatenate([shadow_datasets_test_class_k[i][k*range_indices_test:(k+1)*range_indices_test] for i in range(len(c_s))]))
    shadow_train_labels_k.append(np.concatenate([shadow_train_labels_class_k[i][k*range_indices_train:(k+1)*range_indices_train] for i in range(len(c_s))]))
    shadow_test_labels_k.append(np.concatenate([shadow_test_labels_class_k[i][k*range_indices_test:(k+1)*range_indices_test] for i in range(len(c_s))]))
    shadow_dataset_train_k[k], shadow_train_labels_k[k] = shuffle(shadow_dataset_train_k[k], shadow_train_labels_k[k])

shadow_dataset_train_k

[array([[[4.458402  , 3.8980637 , 1.259603  , 1.9980637 ]],
 
        [[7.8874593 , 4.459631  , 4.800767  , 0.32715416]],
 
        [[7.7245164 , 2.8127816 , 2.9980881 , 0.2348094 ]],
 
        ...,
 
        [[4.3162737 , 3.2702246 , 1.0266708 , 0.11084914]],
 
        [[4.517254  , 3.271134  , 1.3560548 , 0.4511175 ]],
 
        [[5.8588357 , 3.0392237 , 3.5547583 , 0.9727907 ]]], dtype=float32)]

In [16]:
def from_list_to_np(shadow_dataset, shadow_labels):
    "Transofrm a list of tensors into a numpy array"
    shadow_dataset_np = np.zeros((len(shadow_dataset), len(shadow_dataset[0][0])))
    shadow_labels_np = np.zeros(len(shadow_dataset))
    for i in range(len(shadow_dataset)):
        shadow_dataset_np[i] = shadow_dataset[i][0]
        shadow_labels_np[i] = shadow_labels[i]
    return shadow_dataset_np, shadow_labels_np

In [17]:
for k in range(mutliple_of_cs):
    shadow_dataset_train_k[k], shadow_train_labels_k[k] = from_list_to_np(shadow_dataset_train_k[k], shadow_train_labels_k[k])
    shadow_dataset_test_k[k], shadow_test_labels_k[k] = from_list_to_np(shadow_dataset_test_k[k], shadow_test_labels_k[k])

In [18]:
similarity_matrixes = []
Edges = []


In [149]:
for k in range(mutliple_of_cs):
    similarity_matrix = cosine_similarity(shadow_dataset_train_k[k],shadow_dataset_train_k[k])
    similarity_matrixes.append(similarity_matrix)
    edges_k = []
    for i in tqdm(range(len(similarity_matrix))):
        for j in range(i):
            if similarity_matrix[i][j] >= minimum_similarity:
                edges_k.append((i,j))
    Edges.append(edges_k)

KeyboardInterrupt: 

In [ ]:
shadow_model_s = []
for k in range(mutliple_of_cs):
    print(f'Shadow model {k} training...')
    x = torch.tensor(shadow_dataset_train_k[k], dtype=torch.float)

    edge_index = torch.tensor(Edges[k], dtype=torch.long).t().contiguous()
    data = Data(x=x, edge_index=edge_index)
    y_true_tensor = torch.tensor(shadow_train_labels_k[k], dtype=torch.long, requires_grad=False)
    data.y = y_true_tensor
    data.validate(raise_on_error=True)
    print("Data done")
    # Manually split nodes into train, validation, and test sets using PyTorch
    num_nodes = len(data.x)
    train_ratio, test_ratio = 0.8, 0.2

    num_train = int(train_ratio * num_nodes)
    num_test = num_nodes - num_train

    # Create masks for train, validation, and test nodes
    perm = torch.randperm(num_nodes)
    train_mask = perm[:num_train]
    test_mask = perm[num_train:]

    # Apply masks to the data
    data.train_mask = torch.zeros(num_nodes, dtype=torch.bool)
    data.test_mask = torch.zeros(num_nodes, dtype=torch.bool)

    data.train_mask[train_mask] = 1
    data.test_mask[test_mask] = 1
    print("Mask done")

    shadow_model_k = GraphSAGE(data.num_features, hidden_dim=16, out_dim=3)
    optimizer = torch.optim.Adam(shadow_model_k.parameters(), lr=0.01)
    criterion = torch.nn.CrossEntropyLoss()
    # Training loop
    shadow_model_k.train()
    print("Training...")
    for epoch in tqdm(range(101)):
        optimizer.zero_grad()
        out = shadow_model_k(data)
        loss = criterion(out[data.train_mask], y_true_tensor[data.train_mask])
        loss.backward()
        optimizer.step()
        if epoch % 10 == 0:
            print(f'Epoch: {epoch:03d}, Train Loss: {loss:.3f}')

    # Evaluation
    shadow_model_k.eval()
    with torch.no_grad():
        logits = shadow_model_k(data)
        pred = logits.argmax(dim=1)

    # Evaluate accuracy
    accuracy = (pred[data.test_mask] == y_true_tensor[data.test_mask]).sum().item() / data.test_mask.sum().item()
    print(f"Accuracy: {accuracy:.4f}")
    shadow_model_s.append(shadow_model_k)

Shadow model 0 training...
Data done
Mask done
Training...


  1%|          | 1/101 [00:13<21:40, 13.01s/it]

Epoch: 000, Train Loss: 1.896


 11%|█         | 11/101 [02:16<18:23, 12.26s/it]

Epoch: 010, Train Loss: 1.064


...

 90%|█████████ | 91/101 [18:30<02:01, 12.18s/it]

Epoch: 090, Train Loss: 0.047


100%|██████████| 101/101 [20:32<00:00, 12.20s/it]

Epoch: 100, Train Loss: 0.037


Accuracy: 0.9987


In [ ]:
torch.save(shadow_model_s, 'shadow_modelsV0.pt')

In [19]:
shadow_model_s = torch.load('shadow_modelsV0.pt')

In [20]:
def attack_training_set_in(shadow_training_dataset, shadow_train_labels_k, target_shadow_model):
    attack_dataset = []
    attack_labels = []
    print(f'Generating attack dataset in...')
    for k in tqdm(range(len(shadow_training_dataset))):
        if shadow_training_dataset[k] is not torch.tensor:
            x = torch.tensor([shadow_training_dataset[k]], dtype=torch.float, requires_grad=False)
        else :
            x = torch.tensor([shadow_training_dataset[k].type(torch.FloatTensor).detach()])
        edge_index = torch.tensor([(0,0)], dtype=torch.long).t().contiguous()
        data = Data(x=x, edge_index=edge_index)
        y = F.softmax(target_shadow_model.forward(data).detach())
        attack_dataset.append([y[0].numpy(), shadow_train_labels_k[k]])
        attack_labels.append(1)
    return attack_dataset, attack_labels

def attack_training_set_out(shadow_test_dataset, shadow_test_labels_k, target_shadow_model):
    attack_dataset = []
    attack_labels = []
    print(f'Generating attack dataset out...')
    for k in tqdm(range(len(shadow_test_dataset))):
        if shadow_test_dataset[k] is not torch.tensor:
            x = torch.tensor([shadow_test_dataset[k]], dtype=torch.float, requires_grad=False)
        else :
            x = torch.tensor([shadow_test_dataset[k].type(torch.FloatTensor).detach()])
        edge_index = torch.tensor([(0,0)], dtype=torch.long).t().contiguous()
        data = Data(x=x, edge_index=edge_index)
        y = F.softmax(target_shadow_model.forward(data).detach())
        attack_dataset.append([y[0].numpy(), shadow_test_labels_k[k]])
        attack_labels.append(0)
    return attack_dataset, attack_labels

In [21]:
shadow_dataset_test_k[0]

array([[4.35231924, 3.01741147, 1.08574498, 0.13487931],
       [6.35557413, 4.28450727, 1.13643396, 0.15549856],
       [4.36786222, 4.00718021, 1.11121798, 0.96438307],
       ...,
       [5.38039255, 3.56585312, 6.86368847, 1.00048339],
       [4.61980343, 2.62902021, 4.11406517, 1.70622289],
       [4.87551832, 2.20173454, 6.28643703, 1.79689825]])

In [22]:
attack_dataset = []
attack_labels = []
for k in range(mutliple_of_cs):
    attack_dataset_in, attack_labels_in = attack_training_set_in(shadow_dataset_train_k[k], shadow_train_labels_k[k], shadow_model_s[k])
    attack_dataset_out, attack_labels_out = attack_training_set_out(shadow_dataset_test_k[k], shadow_test_labels_k[k], shadow_model_s[k])
    attack_dataset += attack_dataset_in
    attack_dataset += attack_dataset_out
    attack_labels += attack_labels_in
    attack_labels += attack_labels_out
attack_labels = np.array(attack_labels)
print(len(attack_labels))

Generating attack dataset in...


  0%|          | 0/15000 [00:00<?, ?it/s]/var/folders/62/6pb5kjf139gccp8_mvcxrc7w0000gn/T/ipykernel_75829/793026817.py:7: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1682343686209/work/torch/csrc/utils/tensor_new.cpp:248.)
  x = torch.tensor([shadow_training_dataset[k]], dtype=torch.float, requires_grad=False)
/var/folders/62/6pb5kjf139gccp8_mvcxrc7w0000gn/T/ipykernel_75829/793026817.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  y = F.softmax(target_shadow_model.forward(data).detach())
100%|██████████| 15000/15000 [00:02<00:00, 5218.27it/s]


Generating attack dataset out...


  0%|          | 0/15000 [00:00<?, ?it/s]/var/folders/62/6pb5kjf139gccp8_mvcxrc7w0000gn/T/ipykernel_75829/793026817.py:28: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  y = F.softmax(target_shadow_model.forward(data).detach())
100%|██████████| 15000/15000 [00:02<00:00, 5510.11it/s]

30000


In [23]:
import pandas as pd
df = pd.DataFrame(data=attack_dataset, columns=['y', 'class'])
df[df['class'] == 1.0]

,y,class
1,"[0.00990214, 0.79080725, 0.19929065]",1.0
2,"[0.058757894, 0.9347302, 0.0065118284]",1.0
3,"[0.010451914, 0.8305525, 0.15899555]",1.0
5,"[0.009989712, 0.7973351, 0.19267517]",1.0
10,"[0.0101730665, 0.8107749, 0.17905195]",1.0
...,...,...
24995,"[0.010370922, 0.82489705, 0.16473202]",1.0
24996,"[0.01042097, 0.82840085, 0.16117817]",1.0
24997,"[0.010425883, 0.82874286, 0.16083129]",1.0
24998,"[0.01034083, 0.8227778, 0.1668813]",1.0


In [24]:
from sklearn.ensemble import RandomForestClassifier


attack_dataset = []
attack_models = []
for c in c_s:
    dataset = df[df['class'] == c]['y']
    labels = attack_labels[df['class'] == c]
    dataset = np.array([x for x in dataset])
    train_dataset, test_dataset, train_labels, test_labels = train_test_split(dataset, labels, test_size=0.1, shuffle=True)
    print(sum(train_labels == 1))
    print(f'Attack model {c} training...')
    # attack_model = MLPClassifier(hidden_layer_sizes=(80, 70, 60, 50,40, 30, 20,10), max_iter=1000).fit(train_dataset, train_labels)
    attack_model = RandomForestClassifier(n_estimators=1000).fit(train_dataset, train_labels)
    y_pred = attack_model.predict(test_dataset)
    print(f'Attack model {c} accuracy: {attack_model.score(test_dataset, test_labels)}')
    print(f'Attack model {c} recall: {recall_score(test_labels, y_pred)}')
    print(f'Attack model {c} precision: {precision_score(test_labels, y_pred)}')
    print(f'Attack model {c} f1: {f1_score(test_labels, y_pred)}\n')
    attack_models.append(attack_model)

4493
Attack model 0 training...
Attack model 0 accuracy: 0.512
Attack model 0 recall: 0.5108481262327417
Attack model 0 precision: 0.5190380761523046
Attack model 0 f1: 0.5149105367793241

4521
Attack model 1 training...
Attack model 1 accuracy: 0.504
Attack model 1 recall: 0.48434237995824636
Attack model 1 precision: 0.48232848232848236
Attack model 1 f1: 0.4833333333333334

4519
Attack model 2 training...
Attack model 2 accuracy: 0.515
Attack model 2 recall: 0.5239085239085239
Attack model 2 precision: 0.49606299212598426
Attack model 2 f1: 0.5096056622851365



In [25]:
data_target_model_train = data_target_model.x[data_target_model.train_mask]
data_target_model_test = data_target_model.x[data_target_model.test_mask]
y_train_target_model = data_target_model.y[data_target_model.train_mask]
y_test_target_model = data_target_model.y[data_target_model.test_mask]

data_target_model_train_class_0 = data_target_model_train[y_train_target_model == 0]
data_target_model_train_class_1 = data_target_model_train[y_train_target_model == 1]
data_target_model_train_class_2 = data_target_model_train[y_train_target_model == 2]

data_target_model_test_class_0 = data_target_model_test[y_test_target_model == 0]
data_target_model_test_class_1 = data_target_model_test[y_test_target_model == 1]
data_target_model_test_class_2 = data_target_model_test[y_test_target_model == 2]

data_in_sample_0 = Data(x=data_target_model_train_class_0, edge_index=torch.tensor([(0,0)]*len(data_target_model_train_class_0), dtype=torch.long).t().contiguous())
data_in_sample_1 = Data(x=data_target_model_train_class_1, edge_index=torch.tensor([(0,0)]*len(data_target_model_train_class_1), dtype=torch.long).t().contiguous())
data_in_sample_2 = Data(x=data_target_model_train_class_2, edge_index=torch.tensor([(0,0)]*len(data_target_model_train_class_2), dtype=torch.long).t().contiguous())

data_out_sample_0 = Data(x=data_target_model_test_class_0, edge_index=torch.tensor([(0,0)]*len(data_target_model_test_class_0), dtype=torch.long).t().contiguous())
data_out_sample_1 = Data(x=data_target_model_test_class_1, edge_index=torch.tensor([(0,0)]*len(data_target_model_test_class_1), dtype=torch.long).t().contiguous())
data_out_sample_2 = Data(x=data_target_model_test_class_2, edge_index=torch.tensor([(0,0)]*len(data_target_model_test_class_2), dtype=torch.long).t().contiguous())

In_samples_y_0 = F.softmax(target_model.forward(data_in_sample_0).detach())
In_samples_y_1 = F.softmax(target_model.forward(data_in_sample_1).detach())
In_samples_y_2 = F.softmax(target_model.forward(data_in_sample_2).detach())

In_samples_y_0 = np.array([x for x in In_samples_y_0])
In_samples_y_1 = np.array([x for x in In_samples_y_1])
In_samples_y_2 = np.array([x for x in In_samples_y_2])

Out_samples_y_0 = F.softmax(target_model.forward(data_out_sample_0).detach())
Out_samples_y_1 = F.softmax(target_model.forward(data_out_sample_1).detach())
Out_samples_y_2 = F.softmax(target_model.forward(data_out_sample_2).detach())

Out_samples_y_0 = np.array([x for x in Out_samples_y_0])
Out_samples_y_1 = np.array([x for x in Out_samples_y_1])
Out_samples_y_2 = np.array([x for x in Out_samples_y_2])


labels_in_0 = np.full(len(In_samples_y_0), 1)
labels_in_1 = np.full(len(In_samples_y_1), 1)
labels_in_2 = np.full(len(In_samples_y_2), 1)

labels_out_0 = np.full(len(Out_samples_y_0), 0)
labels_out_1 = np.full(len(Out_samples_y_1), 0)
labels_out_2 = np.full(len(Out_samples_y_2), 0)

evaluation_0 = np.concatenate((In_samples_y_0, Out_samples_y_0))
evaluation_1 = np.concatenate((In_samples_y_1, Out_samples_y_1))
evaluation_2 = np.concatenate((In_samples_y_2, Out_samples_y_2))

labels_0 = np.concatenate((labels_in_0, labels_out_0))
labels_1 = np.concatenate((labels_in_1, labels_out_1))
labels_2 = np.concatenate((labels_in_2, labels_out_2))

y_pred_0 = attack_models[0].predict(evaluation_0)
y_pred_1 = attack_models[1].predict(evaluation_1)
y_pred_2 = attack_models[2].predict(evaluation_2)

print(f'Attack model accuracy on class 0: {accuracy_score(labels_0, y_pred_0)}')
print(f'Attack model recall on class 0: {recall_score(labels_0, y_pred_0)}')
print(f'Attack model precision on class 0: {precision_score(labels_0, y_pred_0)}')
print(f'Attack model f1 on class 0: {f1_score(labels_0, y_pred_0)}\n')

print(f'Attack model accuracy on class 1: {accuracy_score(labels_1, y_pred_1)}')
print(f'Attack model recall on class 1: {recall_score(labels_1, y_pred_1)}')
print(f'Attack model precision on class 1: {precision_score(labels_1, y_pred_1)}')
print(f'Attack model f1 on class 1: {f1_score(labels_1, y_pred_1)}\n')

print(f'Attack model accuracy on class 2: {accuracy_score(labels_2, y_pred_2)}')
print(f'Attack model recall on class 2: {recall_score(labels_2, y_pred_2)}')
print(f'Attack model precision on class 2: {precision_score(labels_2, y_pred_2)}')
print(f'Attack model f1 on class 2: {f1_score(labels_2, y_pred_2)}\n')

Attack model accuracy on class 0: 0.62
Attack model recall on class 0: 0.6
Attack model precision on class 0: 0.8888888888888888
Attack model f1 on class 0: 0.7164179104477612

Attack model accuracy on class 1: 0.5
Attack model recall on class 1: 0.5641025641025641
Attack model precision on class 1: 0.7333333333333333
Attack model f1 on class 1: 0.6376811594202899

Attack model accuracy on class 2: 0.28
Attack model recall on class 2: 0.17073170731707318
Attack model precision on class 2: 0.7777777777777778
Attack model f1 on class 2: 0.27999999999999997



/var/folders/62/6pb5kjf139gccp8_mvcxrc7w0000gn/T/ipykernel_75829/1748551869.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  In_samples_y_0 = F.softmax(target_model.forward(data_in_sample_0).detach())
/var/folders/62/6pb5kjf139gccp8_mvcxrc7w0000gn/T/ipykernel_75829/1748551869.py:23: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  In_samples_y_1 = F.softmax(target_model.forward(data_in_sample_1).detach())
/var/folders/62/6pb5kjf139gccp8_mvcxrc7w0000gn/T/ipykernel_75829/1748551869.py:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  In_samples_y_2 = F.softmax(target_model.forward(data_in_sample_2).detach())
/var/folders/62/6pb5kjf139gccp8_mvcxrc7w0000gn/T/ipykernel_75829/1748551869.py:30: UserWarning: Implicit dimension choice for softmax has been deprecated.

In [31]:
y_predicted = target_model.forward(data_target_model).detach()
y_predicted_train = y_predicted[data_target_model.train_mask]
y_predicted_test = y_predicted[data_target_model.test_mask]
class_train = data_target_model.y[data_target_model.train_mask]
class_test = data_target_model.y[data_target_model.test_mask]

y_predicted_train = F.softmax(y_predicted_train)
y_predicted_test = F.softmax(y_predicted_test)

y_predicted_train = np.array([x for x in y_predicted_train])
y_predicted_test = np.array([x for x in y_predicted_test])
class_train = np.array([x for x in class_train])
class_test = np.array([x for x in class_test])

index_train_0 = np.where(class_train == 0)
index_train_1 = np.where(class_train == 1)
index_train_2 = np.where(class_train == 2)

index_test_0 = np.where(class_test == 0)
index_test_1 = np.where(class_test == 1)
index_test_2 = np.where(class_test == 2)

In_samples_y_0 = y_predicted_train[index_train_0]
In_samples_y_1 = y_predicted_train[index_train_1]
In_samples_y_2 = y_predicted_train[index_train_2]

Out_samples_y_0 = y_predicted_test[index_test_0]
Out_samples_y_1 = y_predicted_test[index_test_1]
Out_samples_y_2 = y_predicted_test[index_test_2]


labels_in_0 = np.full(len(In_samples_y_0), 1)
labels_in_1 = np.full(len(In_samples_y_1), 1)
labels_in_2 = np.full(len(In_samples_y_2), 1)

labels_out_0 = np.full(len(Out_samples_y_0), 0)
labels_out_1 = np.full(len(Out_samples_y_1), 0)
labels_out_2 = np.full(len(Out_samples_y_2), 0)

evaluation_0 = np.concatenate((In_samples_y_0, Out_samples_y_0))
evaluation_1 = np.concatenate((In_samples_y_1, Out_samples_y_1))
evaluation_2 = np.concatenate((In_samples_y_2, Out_samples_y_2))

labels_0 = np.concatenate((labels_in_0, labels_out_0))
labels_1 = np.concatenate((labels_in_1, labels_out_1))
labels_2 = np.concatenate((labels_in_2, labels_out_2))

y_pred_0 = attack_models[0].predict(evaluation_0)
y_pred_1 = attack_models[1].predict(evaluation_1)
y_pred_2 = attack_models[2].predict(evaluation_2)

print(f'Attack model accuracy on class 0: {accuracy_score(labels_0, y_pred_0)}')
print(f'Attack model recall on class 0: {recall_score(labels_0, y_pred_0)}')
print(f'Attack model precision on class 0: {precision_score(labels_0, y_pred_0)}')
print(f'Attack model f1 on class 0: {f1_score(labels_0, y_pred_0)}\n')

print(f'Attack model accuracy on class 1: {accuracy_score(labels_1, y_pred_1)}')
print(f'Attack model recall on class 1: {recall_score(labels_1, y_pred_1)}')
print(f'Attack model precision on class 1: {precision_score(labels_1, y_pred_1)}')
print(f'Attack model f1 on class 1: {f1_score(labels_1, y_pred_1)}\n')

print(f'Attack model accuracy on class 2: {accuracy_score(labels_2, y_pred_2)}')
print(f'Attack model recall on class 2: {recall_score(labels_2, y_pred_2)}')
print(f'Attack model precision on class 2: {precision_score(labels_2, y_pred_2)}')
print(f'Attack model f1 on class 2: {f1_score(labels_2, y_pred_2)}\n')


Attack model accuracy on class 0: 0.7
Attack model recall on class 0: 0.85
Attack model precision on class 0: 0.7906976744186046
Attack model f1 on class 0: 0.8192771084337349

Attack model accuracy on class 1: 0.76
Attack model recall on class 1: 0.9743589743589743
Attack model precision on class 1: 0.7755102040816326
Attack model f1 on class 1: 0.8636363636363635

Attack model accuracy on class 2: 0.56
Attack model recall on class 2: 0.5853658536585366
Attack model precision on class 2: 0.8275862068965517
Attack model f1 on class 2: 0.6857142857142856



/var/folders/62/6pb5kjf139gccp8_mvcxrc7w0000gn/T/ipykernel_75829/57496897.py:7: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  y_predicted_train = F.softmax(y_predicted_train)
/var/folders/62/6pb5kjf139gccp8_mvcxrc7w0000gn/T/ipykernel_75829/57496897.py:8: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  y_predicted_test = F.softmax(y_predicted_test)


In [34]:
new_target_model = GraphSAGE(data_target_model.num_features, hidden_dim=16, out_dim=3)
optimizer = torch.optim.Adam(new_target_model.parameters(), lr=0.01)

In [35]:
# Training loop
new_target_model.train()
for epoch in range(11):
    optimizer.zero_grad()
    out = new_target_model(data_target_model)
    loss = F.cross_entropy(out[data_target_model.train_mask], y_true_tensor[data_target_model.train_mask])
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print(f'Epoch: {epoch:03d}, Train Loss: {loss:.3f}')

# Evaluation
new_target_model.eval()
with torch.no_grad():
    logits = new_target_model(data_target_model)
    pred = logits.argmax(dim=1)

# Evaluate accuracy
accuracy = (pred[data_target_model.test_mask] == y_true_tensor[data_target_model.test_mask]).sum().item() / data_target_model.test_mask.sum().item()
print(f"Accuracy: {accuracy:.4f}")

Epoch: 000, Train Loss: 1.228
Epoch: 010, Train Loss: 0.712
Accuracy: 0.6333


In [36]:
y_predicted = new_target_model.forward(data_target_model).detach()
y_predicted_train = y_predicted[data_target_model.train_mask]
y_predicted_test = y_predicted[data_target_model.test_mask]
class_train = data_target_model.y[data_target_model.train_mask]
class_test = data_target_model.y[data_target_model.test_mask]

y_predicted_train = F.softmax(y_predicted_train)
y_predicted_test = F.softmax(y_predicted_test)

y_predicted_train = np.array([x for x in y_predicted_train])
y_predicted_test = np.array([x for x in y_predicted_test])
class_train = np.array([x for x in class_train])
class_test = np.array([x for x in class_test])

index_train_0 = np.where(class_train == 0)
index_train_1 = np.where(class_train == 1)
index_train_2 = np.where(class_train == 2)

index_test_0 = np.where(class_test == 0)
index_test_1 = np.where(class_test == 1)
index_test_2 = np.where(class_test == 2)

In_samples_y_0 = y_predicted_train[index_train_0]
In_samples_y_1 = y_predicted_train[index_train_1]
In_samples_y_2 = y_predicted_train[index_train_2]

Out_samples_y_0 = y_predicted_test[index_test_0]
Out_samples_y_1 = y_predicted_test[index_test_1]
Out_samples_y_2 = y_predicted_test[index_test_2]


labels_in_0 = np.full(len(In_samples_y_0), 1)
labels_in_1 = np.full(len(In_samples_y_1), 1)
labels_in_2 = np.full(len(In_samples_y_2), 1)

labels_out_0 = np.full(len(Out_samples_y_0), 0)
labels_out_1 = np.full(len(Out_samples_y_1), 0)
labels_out_2 = np.full(len(Out_samples_y_2), 0)

evaluation_0 = np.concatenate((In_samples_y_0, Out_samples_y_0))
evaluation_1 = np.concatenate((In_samples_y_1, Out_samples_y_1))
evaluation_2 = np.concatenate((In_samples_y_2, Out_samples_y_2))

labels_0 = np.concatenate((labels_in_0, labels_out_0))
labels_1 = np.concatenate((labels_in_1, labels_out_1))
labels_2 = np.concatenate((labels_in_2, labels_out_2))

y_pred_0 = attack_models[0].predict(evaluation_0)
y_pred_1 = attack_models[1].predict(evaluation_1)
y_pred_2 = attack_models[2].predict(evaluation_2)

print(f'Attack model accuracy on class 0: {accuracy_score(labels_0, y_pred_0)}')
print(f'Attack model recall on class 0: {recall_score(labels_0, y_pred_0)}')
print(f'Attack model precision on class 0: {precision_score(labels_0, y_pred_0)}')
print(f'Attack model f1 on class 0: {f1_score(labels_0, y_pred_0)}\n')

print(f'Attack model accuracy on class 1: {accuracy_score(labels_1, y_pred_1)}')
print(f'Attack model recall on class 1: {recall_score(labels_1, y_pred_1)}')
print(f'Attack model precision on class 1: {precision_score(labels_1, y_pred_1)}')
print(f'Attack model f1 on class 1: {f1_score(labels_1, y_pred_1)}\n')

print(f'Attack model accuracy on class 2: {accuracy_score(labels_2, y_pred_2)}')
print(f'Attack model recall on class 2: {recall_score(labels_2, y_pred_2)}')
print(f'Attack model precision on class 2: {precision_score(labels_2, y_pred_2)}')
print(f'Attack model f1 on class 2: {f1_score(labels_2, y_pred_2)}\n')


Attack model accuracy on class 0: 0.2
Attack model recall on class 0: 0.0
Attack model precision on class 0: 0.0
Attack model f1 on class 0: 0.0

Attack model accuracy on class 1: 0.78
Attack model recall on class 1: 1.0
Attack model precision on class 1: 0.78
Attack model f1 on class 1: 0.8764044943820225

Attack model accuracy on class 2: 0.54
Attack model recall on class 2: 0.5609756097560976
Attack model precision on class 2: 0.8214285714285714
Attack model f1 on class 2: 0.6666666666666667



/var/folders/62/6pb5kjf139gccp8_mvcxrc7w0000gn/T/ipykernel_75829/2638329020.py:7: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  y_predicted_train = F.softmax(y_predicted_train)
/var/folders/62/6pb5kjf139gccp8_mvcxrc7w0000gn/T/ipykernel_75829/2638329020.py:8: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  y_predicted_test = F.softmax(y_predicted_test)
/Users/jskaf/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [37]:
data_target_model_train = data_target_model.x[data_target_model.train_mask]
data_target_model_test = data_target_model.x[data_target_model.test_mask]
y_train_target_model = data_target_model.y[data_target_model.train_mask]
y_test_target_model = data_target_model.y[data_target_model.test_mask]

data_target_model_train_class_0 = data_target_model_train[y_train_target_model == 0]
data_target_model_train_class_1 = data_target_model_train[y_train_target_model == 1]
data_target_model_train_class_2 = data_target_model_train[y_train_target_model == 2]

data_target_model_test_class_0 = data_target_model_test[y_test_target_model == 0]
data_target_model_test_class_1 = data_target_model_test[y_test_target_model == 1]
data_target_model_test_class_2 = data_target_model_test[y_test_target_model == 2]

data_in_sample_0 = Data(x=data_target_model_train_class_0, edge_index=torch.tensor([(0,0)]*len(data_target_model_train_class_0), dtype=torch.long).t().contiguous())
data_in_sample_1 = Data(x=data_target_model_train_class_1, edge_index=torch.tensor([(0,0)]*len(data_target_model_train_class_1), dtype=torch.long).t().contiguous())
data_in_sample_2 = Data(x=data_target_model_train_class_2, edge_index=torch.tensor([(0,0)]*len(data_target_model_train_class_2), dtype=torch.long).t().contiguous())

data_out_sample_0 = Data(x=data_target_model_test_class_0, edge_index=torch.tensor([(0,0)]*len(data_target_model_test_class_0), dtype=torch.long).t().contiguous())
data_out_sample_1 = Data(x=data_target_model_test_class_1, edge_index=torch.tensor([(0,0)]*len(data_target_model_test_class_1), dtype=torch.long).t().contiguous())
data_out_sample_2 = Data(x=data_target_model_test_class_2, edge_index=torch.tensor([(0,0)]*len(data_target_model_test_class_2), dtype=torch.long).t().contiguous())

In_samples_y_0 = F.softmax(new_target_model.forward(data_in_sample_0).detach())
In_samples_y_1 = F.softmax(new_target_model.forward(data_in_sample_1).detach())
In_samples_y_2 = F.softmax(new_target_model.forward(data_in_sample_2).detach())

In_samples_y_0 = np.array([x for x in In_samples_y_0])
In_samples_y_1 = np.array([x for x in In_samples_y_1])
In_samples_y_2 = np.array([x for x in In_samples_y_2])

Out_samples_y_0 = F.softmax(new_target_model.forward(data_out_sample_0).detach())
Out_samples_y_1 = F.softmax(new_target_model.forward(data_out_sample_1).detach())
Out_samples_y_2 = F.softmax(new_target_model.forward(data_out_sample_2).detach())

Out_samples_y_0 = np.array([x for x in Out_samples_y_0])
Out_samples_y_1 = np.array([x for x in Out_samples_y_1])
Out_samples_y_2 = np.array([x for x in Out_samples_y_2])


labels_in_0 = np.full(len(In_samples_y_0), 1)
labels_in_1 = np.full(len(In_samples_y_1), 1)
labels_in_2 = np.full(len(In_samples_y_2), 1)

labels_out_0 = np.full(len(Out_samples_y_0), 0)
labels_out_1 = np.full(len(Out_samples_y_1), 0)
labels_out_2 = np.full(len(Out_samples_y_2), 0)

evaluation_0 = np.concatenate((In_samples_y_0, Out_samples_y_0))
evaluation_1 = np.concatenate((In_samples_y_1, Out_samples_y_1))
evaluation_2 = np.concatenate((In_samples_y_2, Out_samples_y_2))

labels_0 = np.concatenate((labels_in_0, labels_out_0))
labels_1 = np.concatenate((labels_in_1, labels_out_1))
labels_2 = np.concatenate((labels_in_2, labels_out_2))

y_pred_0 = attack_models[0].predict(evaluation_0)
y_pred_1 = attack_models[1].predict(evaluation_1)
y_pred_2 = attack_models[2].predict(evaluation_2)

print(f'Attack model accuracy on class 0: {accuracy_score(labels_0, y_pred_0)}')
print(f'Attack model recall on class 0: {recall_score(labels_0, y_pred_0)}')
print(f'Attack model precision on class 0: {precision_score(labels_0, y_pred_0)}')
print(f'Attack model f1 on class 0: {f1_score(labels_0, y_pred_0)}\n')

print(f'Attack model accuracy on class 1: {accuracy_score(labels_1, y_pred_1)}')
print(f'Attack model recall on class 1: {recall_score(labels_1, y_pred_1)}')
print(f'Attack model precision on class 1: {precision_score(labels_1, y_pred_1)}')
print(f'Attack model f1 on class 1: {f1_score(labels_1, y_pred_1)}\n')

print(f'Attack model accuracy on class 2: {accuracy_score(labels_2, y_pred_2)}')
print(f'Attack model recall on class 2: {recall_score(labels_2, y_pred_2)}')
print(f'Attack model precision on class 2: {precision_score(labels_2, y_pred_2)}')
print(f'Attack model f1 on class 2: {f1_score(labels_2, y_pred_2)}\n')

Attack model accuracy on class 0: 0.2
Attack model recall on class 0: 0.0
Attack model precision on class 0: 0.0
Attack model f1 on class 0: 0.0

Attack model accuracy on class 1: 0.78
Attack model recall on class 1: 1.0
Attack model precision on class 1: 0.78
Attack model f1 on class 1: 0.8764044943820225

Attack model accuracy on class 2: 0.56
Attack model recall on class 2: 0.5853658536585366
Attack model precision on class 2: 0.8275862068965517
Attack model f1 on class 2: 0.6857142857142856



/var/folders/62/6pb5kjf139gccp8_mvcxrc7w0000gn/T/ipykernel_75829/3036034558.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  In_samples_y_0 = F.softmax(new_target_model.forward(data_in_sample_0).detach())
/var/folders/62/6pb5kjf139gccp8_mvcxrc7w0000gn/T/ipykernel_75829/3036034558.py:23: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  In_samples_y_1 = F.softmax(new_target_model.forward(data_in_sample_1).detach())
/var/folders/62/6pb5kjf139gccp8_mvcxrc7w0000gn/T/ipykernel_75829/3036034558.py:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  In_samples_y_2 = F.softmax(new_target_model.forward(data_in_sample_2).detach())
/var/folders/62/6pb5kjf139gccp8_mvcxrc7w0000gn/T/ipykernel_75829/3036034558.py:30: UserWarning: Implicit dimension choice for softmax has been